In [2]:
import numpy as np
import tensorflow as tf
from keras.layers import Dropout
from keras.models import Sequential, load_model
from keras.layers import LSTM, TimeDistributed, Dense, Flatten
from keras.optimizers import SGD
import random
import math
import json
import os
import logging

In [7]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, GRU, Dense, LSTM, TimeDistributed
from keras.optimizers.legacy import SGD
from keras.optimizers.schedules import ExponentialDecay

def GRU_model():

    # 모델 정의
    model = Sequential()

    # GRU 레이어 추가
    model.add(GRU(units=50, return_sequences=True, input_shape=(144, 1), activation='tanh'))
    model.add(GRU(units=50, return_sequences=True, activation='tanh'))

    # 각 타임스텝별 Dense 레이어
    model.add(TimeDistributed(Dense(units=1)))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

In [4]:
def aggregation(local_weights):
    average_weights = []

    # Iterate over each layer index
    for layer_index in range(len(local_weights[0])):
        # Stack the weights of this layer from all models
        layer_weights = np.array([model_weights[layer_index] for model_weights in local_weights])
        
        # Compute the mean of the weights for this layer
        layer_mean = np.mean(layer_weights, axis=0)
        
        # Append the mean weights for this layer to the list
        average_weights.append(layer_mean)

    return average_weights

In [8]:
user_name = ["user1_model", "user2_model", "user3_model"]
weights = []

for i in range(len(user_name)):
    model = f"{user_name[i]}.h5"
    model = load_model(model)    # 모델 로드

    local_weights = model.get_weights()
    weights.append(local_weights)
    
global_weights = aggregation(weights)
global_model = GRU_model()
global_model.set_weights(global_weights)

global_model.save("global_model.h5")


/home/lmls/jisoo/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
